In [2]:
from joblib import dump, load
from sklearn.svm import SVC
from help import helper as hp
from help.utils import Timer
import matplotlib.pyplot as plt
import os
import numpy as np
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

Using TensorFlow backend.


## Ekspresi

In [3]:
expressions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
expressions = np.array(expressions)
expressions

array(['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral'],
      dtype='<U8')

## K-Fold

In [ ]:
# load dataset dan model
kfold = hp.loadjson('dataset/kfold.json')
# models = np.unique([model.split('.')[3] for model in os.listdir('model/dae')])#.................percobaan Semua model
# models = hp.aemodels(cfrom=550, cto=450, cstep=10, n_layer=1)#..................................percobaan 'All'
# models = hp.aemodels(hfrom=1000, hto=1000, hstep=1, cfrom=500, cto=400, cstep=10, n_layer=2)#...percobaan 'Asia Tenggara'
# models = hp.aemodels(cfrom=450, cto=350, cstep=10, n_layer=1)#..................................percobaan 'Asia Timur'
# models = hp.aemodels(hfrom=1000, hto=1000, hstep=1, cfrom=550, cto=450, cstep=10, n_layer=2)#...percobaan 'Eropa Utara'
models = hp.aemodels(cfrom=450, cto=350, cstep=10, n_layer=1)#..................................percobaan 'Amerika Serikat'

dpaths = np.array([f'dataset/vectors/dae/{path}' for path in os.listdir('dataset/vectors/dae')])
scores = {}
scorestime = {}
models
# looping per dataset
# dkeys = list(kfold.keys())
# dkeys = ['all']
# dkeys = ['race Asia Tenggara']
# dkeys = ['race Asia Timur']
# dkeys = ['race Eropa Utara']
dkeys = ['race Amerika Serikat']
for dkey in dkeys:
    # jika ingin learning dengan semua data atau per ras ubah kondisi menjadi (if 'all' in dkey or 'race' in dkey:)
    # jika ingin learning semua dataset ubah kondisi menjadi (if True or 'all' in dkey or 'race' in dkey:)
    if 'all' in dkey or 'race' in dkey:
        scores[dkey] = {'train':{}, 'test':{}}
        scorestime[dkey] = {}
        # looping fold
        for fkey in kfold[dkey]:
            scores[dkey]['train'][fkey] = {}
            scores[dkey]['test'][fkey] = {}
            scorestime[dkey][fkey] = {}
            # looping model
            for model in models:
                clayer = model.split('-')[-1]
                clayer = int(clayer)
                # jika ingin learning dengan layer step 50, ubah kondisi ke (if clayer % 50 == 0:)
                # jika ingin learning semua layer ubah kondisi ke (if True or clayer % 50 == 0:)
                if True or clayer % 50 == 0:
                    # load data training
                    x = []
                    for wholedkey in kfold[dkey][fkey]['train']:
                        train_index = kfold[dkey][fkey]['train'][wholedkey]
                        x.extend(load(f'dataset/vectors/dae/{wholedkey}.{model}.jlb')[train_index])
                    x = np.array(x)
                    y = expressions[x[:,-1].astype(np.int)]
                    x = x[:,:-1]

                    # learning
                    hp.sprint(f'learning ({dkey}|{fkey}|{model})...')
                    timer = Timer().start()
#                     svm = load(f'model/svm/kfold/svm.{dkey}.{fkey}.{model}.jlb')
                    svm = SVC(kernel='linear')
                    svm.fit(x, y)
                    timer.end()
                    dump(svm, f'model/svm/kfold/svm.{dkey}.{fkey}.{model}.jlb')
                    hp.sprint(f'\rlearned ({dkey}|{fkey}|{model}) => {timer.summary(f=2)}')
                    scorestime[dkey][fkey][model] = timer.summary(f=2)

                    #score data training
                    scoretr = np.around(svm.score(x, y) * 100, 2)
                    scores[dkey]['train'][fkey][model] = scoretr
                    hp.sprint(f'\rlearned ({dkey}|{fkey}|{model}|acctr:{scoretr}%) => {timer.summary(f=2)}')

                    # load data testing
                    x = []
                    for wholedkey in kfold[dkey][fkey]['test']:
                        test_index = kfold[dkey][fkey]['test'][wholedkey]
                        x.extend(load(f'dataset/vectors/dae/{wholedkey}.{model}.jlb')[test_index])
                    x = np.array(x)
                    y = expressions[x[:,-1].astype(np.int)]
                    x = x[:,:-1]

                    #score data testing
                    scorett = np.around(svm.score(x, y) * 100, 2)
                    scores[dkey]['test'][fkey][model] = scorett
                    hp.sprint(f'\rlearned ({dkey}|{fkey}|{model}|acctr:{scoretr}%|acctt:{scorett}%) => {timer.summary(f=2)}\n')
    if len(dkeys) == 1:
        break
        
for dkey in scores:
    for typekey in scores[dkey]:
        scores[dkey][typekey]['mean'] = {}
        for model in models:
            clayer = model.split('-')[-1]
            clayer = int(clayer)
            if True or clayer % 50 == 0:
                # mean per model
                mpm = []
                for fkey in range(5):
                    fkey = f'Fold {fkey}'
                    score = scores[dkey][typekey][fkey][model]
                    mpm.append(score)
                mpm = np.around(np.array(mpm).mean(), 2)
                scores[dkey][typekey]['mean'][model] = mpm

if len(dkeys) == 1:
    hp.savejson(f'score/scores_{dkey}.json', scores)
    hp.savejson(f'score/scorestime_{dkey}.json', scorestime)
    hp.savejson(f'score/scoreswithmean_{dkey}.json', scores)
else:
    hp.savejson(f'score/scores.json', scores)
    hp.savejson(f'score/scorestime.json', scorestime)
    hp.savejson(f'score/scoreswithmean.json', scores)
print('done...')

## Learning semua data

In [7]:
race = {
    'asia tenggara': ['1000-410', 'hijrah'],
    'asia timur': ['430', 'jaffe', 'caspeal', 'nvie'],
    'eropa utara': ['1000-550', 'kdef'],
    'amerika serikat': ['400', 'ck+', 'facesdb'],
    'all': ['550', 'all'],
}

for ras in race:
    model = race[ras][0]
    dataset = race[ras][1:]
    
    # load dataset
    x = []
    for name in dataset:
        x.extend(load(f'dataset/vectors/dae/{name}.{model}.jlb'))
    x = np.array(x)
    y = expressions[x[:,-1].astype(np.int)]
    x = x[:,:-1]
    
    # learning
    timer = Timer().start()
    svm = SVC(kernel='linear')
    svm.fit(x, y)
    timer.end()
    dump(svm, f'model/svm/all/{ras}.jlb')
    print(f'{ras} | {timer.summary(f=2)}')
    
    del x
    del y

asia tenggara | 0.5 detik
asia timur | 14.95 detik
eropa utara | 0.27 detik
amerika serikat | 0.55 detik
all | 44.83 detik
